In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
L = 10.0           # Domain length
N = 100            # Number of grid points
Δx = L / N         # Grid spacing
c_high = 1.0       # High concentration
c_low = 0.0        # Low concentration

# Initialize concentration profile
c = np.zeros(N)
for i in range(N):
    if i < N // 2:
        c[i] = c_high
    else:
        c[i] = c_low

# Ensure periodic boundary conditions
c[0] = c[N-1]

# Plot the initial profile
plt.figure(figsize=(8,4))
plt.plot(np.linspace(0, L, N), c, label="Initial Square Wave")
plt.xlabel("x")
plt.ylabel("c(x, 0)")
plt.title("Initial Square Wave Profile")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
L = 10.0           # Domain length
N = 100            # Number of grid points
Δx = L / N         # Grid spacing
c_high = 1.0       # High concentration
c_low = 0.0        # Low concentration

# Initialize concentration profile
c = np.zeros(N)
for i in range(N):
    if i > N // 4 and i <= 3*N // 4:
        c[i] = c_high
    else:
        c[i] = c_low

# Ensure periodic boundary conditions
c[0] = c[N-1]

# Plot the initial profile
plt.figure(figsize=(8,4))
plt.plot(np.linspace(0, L, N), c, label="Initial Square Wave")
plt.xlabel("x")
plt.ylabel("c(x, 0)")
plt.title("Initial Square Wave Profile")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
L = 10.0           # Domain length
N = 100            # Number of grid points
dx = L / N         # Grid spacing
c_high = 1.0       # High concentration
c_low = 0.0        # Low concentration

x = np.linspace(0, L, N, endpoint=False)
np.where((x> N//4) & (x <= 3*N // 4), c_high, c_low)

# Ensure periodic boundary conditions
c[0] = c[N-1]

# Plot the initial profile
plt.figure(figsize=(8,4))
plt.plot(np.linspace(0, L, N), c, label="Initial Square Wave")
plt.xlabel("x")
plt.ylabel("c(x, 0)")
plt.title("Initial Square Wave Profile")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ## Parameters
L = 128        # Domain length
N = 256        # Number of grid points
dx = L / N     # Grid spacing
dt = 1e-6      # Time step (explicit stability constraint)
M = 1.0        # Constant mobility
kappa = 1.0    # Gradient energy coefficient
A = 1.0        # Double-well energy coefficient
T_final = 1  # Total simulation time
plot_intervals = 100  # Number of intermediate plots
interval_step = T_final / plot_intervals  # Time interval for plotting


# ## Define spatial grid
x = np.linspace(0, L, N, endpoint=False)

# ## Smoother Initial Condition Using `tanh`
interface_width = 1.0  # Controls how gradual the transition is
c = 0.5 * (1 + np.tanh((x - L/4) / interface_width) * np.tanh((3*L/4 - x) / interface_width))

# ## Finite Difference Operators with Periodic Boundary Conditions
def laplacian(f, dx):
    """Compute the Laplacian using central finite differences with periodic boundary conditions."""
    return (np.roll(f, -1) - 2*f + np.roll(f, 1)) / dx**2

# ## Time-Stepping Loop
t = 0
time_points = [0]  # Track time steps for plotting
concentration_snapshots = [c.copy()]  # Store snapshots for visualization

while t < T_final:
    # Ensure c is within physical limits [0,1]
    c = np.clip(c, 0.0, 1.0)
    
    # Compute chemical potential
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian(c, dx)
    
    # Compute flux divergence (constant mobility)
    div_J = M * laplacian(mu, dx)

    # Update concentration field using explicit Euler step
    c += dt * div_J

    # Store snapshots at regular intervals
    if t >= time_points[-1] + interval_step:
        time_points.append(t)
        concentration_snapshots.append(c.copy())

    # Update time
    t += dt

# ## Plot Initial, Intermediate, and Final Concentration Profiles
plt.figure(figsize=(8, 6))
for i, (time, snapshot) in enumerate(zip(time_points, concentration_snapshots)):
    plt.plot(x, snapshot, '-o', markersize=2, label=f"t = {time:.2e}")  # Line + Points

plt.xlabel("x")
plt.ylabel("c(x, t)")
plt.title("Cahn–Hilliard Evolution (Constant Mobility, Explicit Scheme)")
plt.legend()
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

#-- Start timing:
start_time = time.time()

#--- Simulation cell parameters:
Nx = 128
dx = 1.0

#--- Time integration parameters:
nstep = 600
nprint = 200
dtime = 0.2

#-- Initialize temperature field & grid:
u0 = np.zeros(Nx)
x = np.arange(1, Nx + 1) * dx

# Set initial condition in the center region
u0[43:84] = 1.0

#-- Evolve temperature field:
ncount = 0
plt.figure(figsize=(10, 6))

for istep in range(1, nstep + 1):
    # Update temperature using finite-difference scheme
    u0[1:-1] += dtime * (u0[2:] - 2.0 * u0[1:-1] + u0[:-2]) / (dx * dx)

    #-- Display results:
    if istep % nprint == 0 or istep == 1:
        ncount += 1
        plt.subplot(2, 2, ncount)
        plt.plot(x, u0)
        plt.title(f'time {istep}')
        plt.axis([0, Nx, -0.5, 1.5])
        plt.xlabel('x')
        plt.ylabel('Temperature')
        if ncount == 4:  # Show 4 subplots
            break

# Finalize plots
plt.tight_layout()
plt.show()

# Compute and display execution time
compute_time = time.time() - start_time
print(f'Compute Time: {compute_time:.2f} seconds')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numba import njit, prange 

# In Numba, prange is a special function similar to Python’s built-in range, 
# but it is used to indicate that the loop can be executed in parallel. 
# When you decorate a function with @njit(parallel=True), 
# loops written with prange are candidates for parallel execution. 
# This can greatly speed up computations on multi-core processors, 
# especially in numerical simulations with independent iterations.

#from numba import njit, prange
#import numpy as np

#@njit(parallel=True)
#def compute_square(arr):
#    result = np.empty_like(arr)
#    for i in prange(len(arr)):
#        result[i] = arr[i] ** 2
#    return result



# ## Parameters
L = 128.0        # Domain length
N = 256         # Number of grid points
dx = L / N       # Grid spacing
dt = 1e-5        # Time step (optimized)
M = 1.0          # Constant mobility
kappa = 1.0      # Gradient energy coefficient
A = 1.0          # Double-well energy coefficient
T_final = 100  # Total simulation time
plot_intervals = 100  # Number of frames in the animation
interval_step = T_final / plot_intervals  # Time interval for storing frames

# ## Define spatial grid
x = np.linspace(0, L, N, endpoint=False)

# ## Initial Condition: 0.5 with small cosine perturbation
perturbation_amplitude = 0.01  # Small amplitude for initial perturbation
perturbation_wavelength = 4    # Number of oscillations in domain
c = 0.5 + perturbation_amplitude * np.cos(2 * np.pi * perturbation_wavelength * x / L)

# ## Finite Difference Operators with Periodic Boundary Conditions
@njit(parallel=True, fastmath=True)
def laplacian(f, dx):
    """Compute the Laplacian using central finite differences with periodic boundary conditions."""
    lap = np.empty_like(f)
    N = len(f)
    for i in prange(N):
        i_plus = (i + 1) % N
        i_minus = (i - 1 + N) % N
        lap[i] = (f[i_plus] - 2*f[i] + f[i_minus]) / dx**2
    return lap

# ## Time-Stepping Loop (Optimized Explicit Euler)
t = 0
time_points = [0]  # Track time steps for animation
concentration_snapshots = [c.copy()]  # Store snapshots for animation

while t < T_final:
    # Ensure c is within physical limits [0,1]
    c = np.clip(c, 0.0, 1.0)
    
    # Compute chemical potential
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian(c, dx)
    
    # Compute flux divergence (constant mobility)
    div_J = M * laplacian(mu, dx)

    # Update concentration field using explicit Euler step
    c += dt * div_J

    # Store snapshots at regular intervals for animation
    if t >= time_points[-1] + interval_step:
        time_points.append(t)
        concentration_snapshots.append(c.copy())

    # Update time
    t += dt

# ## Set up animation
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot(x, concentration_snapshots[0], '-o', markersize=3, label="t = 0")
ax.set_xlabel("x")
ax.set_ylabel("c(x, t)")
ax.set_title("Cahn–Hilliard Evolution (Optimized with Numba)")
ax.set_xlim(0, L)
ax.set_ylim(0, 1)
ax.legend()
ax.grid()

# Animation update function
def update(frame):
    line.set_ydata(concentration_snapshots[frame])
    ax.legend([f"t = {time_points[frame]:.2e}"])
    return line,

# Create animation
ani = animation.FuncAnimation(fig, update, frames=len(concentration_snapshots), interval=50, blit=True)

# Display animation in Jupyter Notebook
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from numba import njit

# ## Parameters
L = 128.0        # Domain length
N = 256         # Number of grid points
dx = L / N       # Grid spacing
dt = 1e-5        # Time step
M = 1.0          # Constant mobility
kappa = 1.0      # Gradient energy coefficient
A = 1.0          # Double-well energy coefficient
T_final = 500    # Total simulation time
plot_intervals = 100  # Number of frames in the animation
interval_step = T_final / plot_intervals  # Time interval for storing frames

# ## Define spatial grid
x = np.linspace(0, L, N, endpoint=False)

# ## Initial Condition: 0.5 with small cosine perturbation
perturbation_amplitude = 0.01  # Small amplitude for initial perturbation
perturbation_wavelength = 4    # Number of oscillations in domain
c0 = 0.5 + perturbation_amplitude * np.cos(2 * np.pi * perturbation_wavelength * x / L)

# ## Precompute Periodic Index Arrays
ip = np.empty(N, dtype=np.int64)
im = np.empty(N, dtype=np.int64)
for i in range(N):
    ip[i] = (i + 1) % N
    im[i] = (i - 1 + N) % N

@njit
def simulate(c, dx, dt, T_final, A, kappa, M, snapshot_interval, ip, im, snapshots):
    t = 0.0
    snap_idx = 0
    # Record the initial condition
    snapshots[snap_idx, :] = c.copy()
    next_snap_t = snapshot_interval
    inv_dx2 = 1.0 / (dx * dx)
    
    while t < T_final:
        # Enforce physical limits on c (manual clipping)
        for i in range(c.size):
            if c[i] < 0.0:
                c[i] = 0.0
            elif c[i] > 1.0:
                c[i] = 1.0

        # Compute Laplacian of c
        lap_c = np.empty_like(c)
        for i in range(c.size):
            lap_c[i] = (c[ip[i]] - 2.0 * c[i] + c[im[i]]) * inv_dx2

        # Compute chemical potential mu
        mu = np.empty_like(c)
        for i in range(c.size):
            mu[i] = 2 * A * c[i] * (1 - c[i]) * (1 - 2 * c[i]) - 2 * kappa * lap_c[i]

        # Compute Laplacian of mu (flux divergence)
        lap_mu = np.empty_like(mu)
        for i in range(mu.size):
            lap_mu[i] = (mu[ip[i]] - 2.0 * mu[i] + mu[im[i]]) * inv_dx2

        # Update concentration field (Explicit Euler)
        for i in range(c.size):
            c[i] += dt * M * lap_mu[i]

        t += dt

        # Store snapshot at regular intervals
        if t >= next_snap_t:
            snap_idx += 1
            snapshots[snap_idx, :] = c.copy()
            next_snap_t += snapshot_interval

    return snapshots

# Preallocate snapshot storage: (number of snapshots) x (grid size)
n_snapshots = plot_intervals + 1
snapshots = np.empty((n_snapshots, N))
c = c0.copy()

# Run the simulation (the time-stepping loop is compiled with Numba)
snapshots = simulate(c, dx, dt, T_final, A, kappa, M, interval_step, ip, im, snapshots)

# ## Set up animation
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot(x, snapshots[0], '-o', markersize=3, label="t = 0")
ax.set_xlabel("x")
ax.set_ylabel("c(x, t)")
ax.set_title("Cahn–Hilliard Evolution (Optimized with Numba)")
ax.set_xlim(0, L)
ax.set_ylim(0, 1)
ax.legend()
ax.grid()

def update(frame):
    line.set_ydata(snapshots[frame])
    ax.legend([f"t = {frame * interval_step:.2e}"])
    return line,

ani = animation.FuncAnimation(fig, update, frames=n_snapshots, interval=50, blit=True)

# Display animation (e.g., in Jupyter Notebook)
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ================================================================
# Step 1: Define Simulation Parameters
# ================================================================
L = 128.0          # Domain length (size of the spatial domain)
N = 256            # Number of grid points (resolution in space)
dx = L / N         # Spatial grid spacing
dt = 1e-5          # Time step size (how much time advances per update)
M = 1.0            # Mobility (controls how fast concentration changes)
kappa = 1.0        # Gradient energy coefficient (related to interface energy)
A = 1.0            # Double-well energy coefficient (controls free energy landscape)
T_final = 100      # Total simulation time
plot_intervals = 100  # Number of snapshots (frames) to store for animation
interval_step = T_final / plot_intervals  # Time between each stored snapshot

# ================================================================
# Step 2: Set Up the Spatial Grid
# ================================================================
x = np.linspace(0, L, N, endpoint=False)  # Create an array of x positions

# ================================================================
# Step 3: Define the Initial Condition
# ================================================================
# Start with a uniform concentration of 0.5 and add a small cosine perturbation.
perturbation_amplitude = 0.01  # Amplitude of the initial perturbation
perturbation_wavelength = 4    # Number of oscillations over the domain
c = 0.5 + perturbation_amplitude * np.cos(2 * np.pi * perturbation_wavelength * x / L)

# ================================================================
# Step 4: Define a Function to Compute the Laplacian
# ================================================================
def laplacian(f, dx):
    """
    Compute the Laplacian of a function f using central finite differences.
    Periodic boundary conditions are applied: the grid "wraps around".
    """
    lap = np.empty_like(f)  # Create an empty array to hold the Laplacian values
    N = len(f)            # Total number of grid points
    for i in range(N):
        # Find indices for the right and left neighbors (with wrapping)
        i_plus = (i + 1) % N   # Right neighbor (wraps to 0 when i is last index)
        i_minus = (i - 1) % N  # Left neighbor (wraps to last index when i is 0)
        lap[i] = (f[i_plus] - 2 * f[i] + f[i_minus]) / dx**2
    return lap

# ================================================================
# Step 5: Time-Stepping Loop to Evolve the System
# ================================================================
# We will store snapshots of the concentration field for later animation.
time_points = [0.0]            # List to record the times at which snapshots are saved
concentration_snapshots = [c.copy()]  # List to store snapshots (copies of c)

t = 0.0  # Start time

while t < T_final:
    # 5a. Enforce physical limits: concentration should be between 0 and 1.
    # np.clip ensures values stay within [0, 1].
    c = np.clip(c, 0.0, 1.0)

    # 5b. Compute the chemical potential mu.
    # The chemical potential here has two parts:
    #   - A local term derived from the double-well free energy.
    #   - A gradient term involving the Laplacian of c.
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian(c, dx)

    # 5c. Compute the divergence of the flux.
    # The flux divergence is given by the Laplacian of mu, scaled by mobility M.
    flux_divergence = M * laplacian(mu, dx)

    # 5d. Update the concentration using an explicit Euler step.
    # c(t+dt) = c(t) + dt * (flux divergence)
    c = c + dt * flux_divergence

    # 5e. Advance the time.
    t += dt

    # 5f. Store snapshots at regular intervals for animation.
    if t >= time_points[-1] + interval_step:
        time_points.append(t)
        concentration_snapshots.append(c.copy())

# ================================================================
# Step 6: Set Up the Animation to Visualize the Evolution
# ================================================================
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot(x, concentration_snapshots[0], '-o', markersize=3,
                label=f"t = {time_points[0]:.2e}")
ax.set_xlabel("x")
ax.set_ylabel("Concentration c(x, t)")
ax.set_title("Cahn–Hilliard Evolution")
ax.set_xlim(0, L)
ax.set_ylim(0, 1)
ax.legend()
ax.grid()

def update(frame):
    """
    Update function for the animation.
    For each frame, update the y-data of the line plot and change the legend.
    """
    line.set_ydata(concentration_snapshots[frame])
    ax.legend([f"t = {time_points[frame]:.2e}"])
    return line,

# Create the animation object.
ani = animation.FuncAnimation(fig, update, frames=len(concentration_snapshots),
                              interval=50, blit=True)

# ================================================================
# Step 7: Display the Animation
# ================================================================
# In a Jupyter Notebook, you can display the animation using HTML:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# ================================================================
# Step 1: Define Simulation Parameters
# ================================================================
L = 128.0          # Domain length (size of the spatial domain)
N = 256            # Number of grid points (resolution in space)
dx = L / N         # Spatial grid spacing
dt = 1e-5          # Time step size (how much time advances per update)
M = 1.0            # Mobility (controls how fast concentration changes)
kappa = 1.0        # Gradient energy coefficient (related to interface energy)
A = 1.0            # Double-well energy coefficient (controls free energy landscape)
T_final = 100      # Total simulation time
plot_intervals = 100  # Number of snapshots (frames) to store for animation
interval_step = T_final / plot_intervals  # Time between each stored snapshot

# ================================================================
# Step 2: Set Up the Spatial Grid
# ================================================================
x = np.linspace(0, L, N, endpoint=False)  # Create an array of x positions

# ================================================================
# Step 3: Define the Initial Condition
# ================================================================
# Start with a uniform concentration of 0.5 and add a small cosine perturbation.
perturbation_amplitude = 0.01  # Amplitude of the initial perturbation
perturbation_wavelength = 4    # Number of oscillations over the domain
c = 0.5 + perturbation_amplitude * np.cos(2 * np.pi * perturbation_wavelength * x / L)

# ================================================================
# Step 4: Define a Function to Compute the Laplacian
# ================================================================
def laplacian_roll(f, dx):
    """
    Compute the Laplacian of a function f using central finite differences.
    Periodic boundary conditions are implemented using np.roll.
    """
    return (np.roll(f, -1) - 2 * f + np.roll(f, 1)) / dx**2

# ================================================================
# Step 5: Time-Stepping Loop to Evolve the System
# ================================================================
# We will store snapshots of the concentration field for later animation.
time_points = [0.0]            # List to record the times at which snapshots are saved
concentration_snapshots = [c.copy()]  # List to store snapshots (copies of c)

t = 0.0  # Start time

while t < T_final:
    # 5a. Enforce physical limits: concentration should be between 0 and 1.
    # np.clip ensures values stay within [0, 1].
    c = np.clip(c, 0.0, 1.0)

    # 5b. Compute the chemical potential mu.
    # The chemical potential here has two parts:
    #   - A local term derived from the double-well free energy.
    #   - A gradient term involving the Laplacian of c.
    #  mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian(c, dx)
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian_roll(c, dx)


    # 5c. Compute the divergence of the flux.
    # The flux divergence is given by the Laplacian of mu, scaled by mobility M.
    # flux_divergence = M * laplacian(mu, dx)
    flux_divergence = M * laplacian_roll(mu, dx)
    
    # 5d. Update the concentration using an explicit Euler step.
    # c(t+dt) = c(t) + dt * (flux divergence)
    c = c + dt * flux_divergence

    # 5e. Advance the time.
    t += dt

    # 5f. Store snapshots at regular intervals for animation.
    if t >= time_points[-1] + interval_step:
        time_points.append(t)
        concentration_snapshots.append(c.copy())

# ================================================================
# Step 6: Set Up the Animation to Visualize the Evolution
# ================================================================
fig, ax = plt.subplots(figsize=(8, 6))
line, = ax.plot(x, concentration_snapshots[0], '-o', markersize=3,
                label=f"t = {time_points[0]:.2e}")
ax.set_xlabel("x")
ax.set_ylabel("Concentration c(x, t)")
ax.set_title("Cahn–Hilliard Evolution")
ax.set_xlim(0, L)
ax.set_ylim(0, 1)
ax.legend()
ax.grid()

def update(frame):
    """
    Update function for the animation.
    For each frame, update the y-data of the line plot and change the legend.
    """
    line.set_ydata(concentration_snapshots[frame])
    ax.legend([f"t = {time_points[frame]:.2e}"])
    return line,

# Create the animation object.
ani = animation.FuncAnimation(fig, update, frames=len(concentration_snapshots),
                              interval=50, blit=True)

# ================================================================
# Step 7: Display the Animation
# ================================================================
# In a Jupyter Notebook, you can display the animation using HTML:
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# ================================================================
# Step 1: Define Simulation Parameters for a 64x64 system
# ================================================================
N = 64                # Number of grid points in each dimension (64x64 system)
dx = 1.0              # Grid spacing in both x and y directions (dx = dy = 0.5)
L = N * dx            # Domain length (L = 64*0.5 = 32)
dt = 1e-2             # Time step size
steps = 100000       # Total number of time steps


M = 1.0               # Mobility
kappa = 1.0           # Gradient energy coefficient
A = 1.0               # Double-well energy coefficient


# How often to store snapshots
plot_intervals = 500  
interval_step = steps // plot_intervals  

# ================================================================
# Step 2: Set Up the 2D  Grid
# ================================================================
x = np.linspace(0, L, N, endpoint=False)
y = np.linspace(0, L, N, endpoint=False)
X, Y = np.meshgrid(x, y)  # Create a 2D grid for visualization

# ================================================================
# Step 3: Define the Initial Condition in 2D
# ================================================================
# Start with a uniform concentration of 0.5 and add a small cosine perturbation.
perturbation_amplitude = 0.01  # Amplitude of the initial perturbation
perturbation_wavelength = 4    # Number of oscillations over the domain

c = 0.5 + perturbation_amplitude * np.cos(2 * np.pi * perturbation_wavelength * X / L) \
          * np.cos(2 * np.pi * perturbation_wavelength * Y / L)

# ================================================================
# Step 4: Define a Function to Compute the 2D Laplacian using np.roll
# ================================================================
def laplacian2d(f, dx):
    """
    Compute the Laplacian of a 2D array f using central finite differences.
    Periodic boundary conditions are implemented using np.roll.
    
    The Laplacian is given by:
      lap(f)[i,j] = (f[i+1,j] + f[i-1,j] + f[i,j+1] + f[i,j-1] - 4*f[i,j]) / dx^2
    """
    return (np.roll(f, -1, axis=0) + np.roll(f, 1, axis=0) +
            np.roll(f, -1, axis=1) + np.roll(f, 1, axis=1) - 4 * f) / dx**2

# ================================================================
# Step 5: Time-Stepping Loop to Evolve the System
# ================================================================
snapshots = []            # List to store snapshots of the concentration field
snapshots.append(c.copy())  # Store the initial condition

# Run the simulation for 10,000 steps
for step in range(steps):
    # Enforce physical bounds: keep c between 0 and 1.
    # c = np.clip(c, 0.0, 1.0)
    
    # Compute the chemical potential mu:
    #   - The local term comes from the double-well potential.
    #   - The gradient term involves the Laplacian of c.
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian2d(c, dx)
    
    # Compute the divergence of the flux (using the Laplacian of mu)
    flux_divergence = M * laplacian2d(mu, dx)
    
    # Update the concentration field using an explicit Euler step.
    c = c + dt * flux_divergence
    
    # Store snapshots at regular intervals for animation.
    if step % interval_step == 0:
        snapshots.append(c.copy())

# ================================================================
# Step 6: Set Up the Animation to Visualize the 2D Evolution
# ================================================================
fig, ax = plt.subplots(figsize=(8,4))
im = ax.imshow(snapshots[0], extent=[0, L, 0, L],
               origin='lower', cmap='viridis')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Cahn–Hilliard Evolution at step 0")
fig.colorbar(im, ax=ax)


def update(frame):
    data = snapshots[frame]
    im.set_array(data)
    # Optionally update the color limits based on the current data:
    im.set_clim(data.min(), data.max())
    ax.set_title(f"Cahn–Hilliard Evolution at step {frame * interval_step}")
    return im,


ani = animation.FuncAnimation(fig, update, frames=len(snapshots),
                              interval=50, blit=True)

# ================================================================
# Step 7: Display the Animation (Jupyter Notebook)
# ================================================================
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
from IPython.display import HTML

# ================================================================
# Step 1: Define Simulation Parameters for a 64x64 system
# ================================================================
N = 128                # Number of grid points in each dimension (64x64 system)
dx = 1.0             # Grid spacing in both x and y directions
L = N * dx            # Domain length (e.g., 64)
dt = 1e-2             # Time step size
steps = 100000        # Total number of time steps

M = 1.0               # Mobility
kappa = 0.5           # Gradient energy coefficient
A = 1.0               # Double-well energy coefficient

# How often to store snapshots for animation
plot_intervals = 500  
interval_step = steps // plot_intervals  

# animation parameters
mpl.rcParams['animation.embed_limit'] = 100 * 1024 * 1024  # 100 MB

# ================================================================
# Step 2: Set Up the 2D Grid
# ================================================================
x = np.linspace(0, L, N, endpoint=False)
y = np.linspace(0, L, N, endpoint=False)
X, Y = np.meshgrid(x, y)  # Create a 2D grid for visualization

# ================================================================
# Step 3: Define the Initial Condition in 2D with Random Conserved Perturbations
# ================================================================


# Define the average composition (c0) and add random perturbations with zero mean. That is conservation of mass

# Always use seed for reproducibility
np.random.seed(714)

c0 = 0.45  # Average initial composition (can be any value between 0 and 1)

perturbation_amplitude = 0.01  # Amplitude of the random perturbations

# Generate random noise in the range [-0.5, 0.5] scaled by perturbation_amplitude.

noise = perturbation_amplitude * (np.random.rand(N, N) - 0.5)
# Subtract the mean of the noise so that the overall mass is conserved (zero mean noise)
noise -= noise.mean()

# The initial composition is the average composition plus the conserved perturbations.
c = c0 + noise

# ================================================================
# Step 4: Define a Function to Compute the 2D Laplacian using np.roll
# ================================================================
def laplacian2d(f, dx):
    """
    Compute the Laplacian of a 2D array f using central finite differences.
    Periodic boundary conditions are implemented using np.roll.
    
    The Laplacian is given by:
      lap(f)[i,j] = (f[i+1,j] + f[i-1,j] + f[i,j+1] + f[i,j-1] - 4*f[i,j]) / dx^2
    """
    return (np.roll(f, -1, axis=0) + np.roll(f, 1, axis=0) +
            np.roll(f, -1, axis=1) + np.roll(f, 1, axis=1) - 4 * f) / dx**2

# ================================================================
# Step 5: Time-Stepping Loop to Evolve the System
# ================================================================
snapshots = []            # List to store snapshots of the concentration field
snapshots.append(c.copy())  # Store the initial condition

for step in range(steps):
    # (a) Compute the chemical potential mu:
    #     - The local term is from the double-well potential.
    #     - The gradient term involves the Laplacian of c.
    mu = 2 * A * c * (1 - c) * (1 - 2 * c) - 2 * kappa * laplacian2d(c, dx)
    
    # (b) Compute the divergence of the flux (using the Laplacian of mu)
    flux_divergence = M * laplacian2d(mu, dx)
    
    # (c) Update the concentration field using an explicit Euler step.
    c = c + dt * flux_divergence
    
    # (d) Store snapshots at regular intervals for animation.
    if step % interval_step == 0:
        snapshots.append(c.copy())

# ================================================================
# Step 6: Set Up the Animation to Visualize the 2D Evolution
# ================================================================
fig, ax = plt.subplots(figsize=(8, 4))
im = ax.imshow(snapshots[0], extent=[0, L, 0, L],
               origin='lower', cmap='jet')
#im = ax.imshow(snapshots[0], extent=[0, L, 0, L], origin='lower', cmap='plasma')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("Cahn–Hilliard Evolution at step 0")
fig.colorbar(im, ax=ax)

def update(frame):
    data = snapshots[frame]
    im.set_array(data)
    # Optionally update the color limits based on the current data:
    im.set_clim(data.min(), data.max())
    ax.set_title(f"Cahn–Hilliard Evolution at step {frame * interval_step}")
    return im,

ani = animation.FuncAnimation(fig, update, frames=len(snapshots),
                              interval=50, blit=True)

# ================================================================
# Step 7: Display the Animation
# ================================================================
HTML(ani.to_jshtml())


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# --- Functions (as defined previously) ---

def initialize_microstructure(Nx, Ny, c0, noise=0.02):
    """
    Initialize the microstructure with random noise.
    """
    return c0 + noise * (np.random.rand(Nx, Ny) - 0.5)

def compute_free_energy_derivative(comp, A=1.0):
    """
    Calculate the derivative of the free energy density.
    """
    return A * (2.0 * comp * (1 - comp)**2 - 2.0 * comp**2 * (1 - comp))

def calculate_energy(comp, grad_coef, dx, dy):
    """
    Calculate the total energy of the system.
    """
    grad_x = (np.roll(comp, -1, axis=0) - comp) / dx
    grad_y = (np.roll(comp, -1, axis=1) - comp) / dy
    energy_density = comp**2 * (1.0 - comp)**2 + 0.5 * grad_coef * (grad_x**2 + grad_y**2)
    return np.sum(energy_density)

def evolve_phase_field(comp, mobility, grad_coef, dtime, dx, dy):
    """
    Perform one time step of the phase field evolution using the Cahn-Hilliard equation.
    """
    # Compute Laplacian of the composition field
    lap_comp = (
        np.roll(comp, -1, axis=0) + np.roll(comp, 1, axis=0) +
        np.roll(comp, -1, axis=1) + np.roll(comp, 1, axis=1) -
        4.0 * comp
    ) / (dx * dy)

    # Compute derivative of free energy
    dfdcomp = compute_free_energy_derivative(comp)

    # Compute chemical potential and its Laplacian
    chem_potential = dfdcomp - grad_coef * lap_comp
    lap_chem_potential = (
        np.roll(chem_potential, -1, axis=0) + np.roll(chem_potential, 1, axis=0) +
        np.roll(chem_potential, -1, axis=1) + np.roll(chem_potential, 1, axis=1) -
        4.0 * chem_potential
    ) / (dx * dy)

    # Update composition field
    comp += dtime * mobility * lap_chem_potential

    # Bound composition to avoid instability
    np.clip(comp, 0.00001, 0.9999, out=comp)

    return comp

def write_vtk(filename, comp, dx, dy):
    """
    Write the composition field to a VTK file for visualization.
    (This section is kept in comments for now.)
    """
    nx, ny = comp.shape
    with open(filename, "w") as f:
        f.write("# vtk DataFile Version 2.0\n")
        f.write("Phase field data\n")
        f.write("ASCII\n")
        f.write("DATASET STRUCTURED_GRID\n")
        f.write(f"DIMENSIONS {nx} {ny} 1\n")
        f.write(f"POINTS {nx * ny} float\n")
        for i in range(nx):
            for j in range(ny):
                f.write(f"{i * dx:.6e} {j * dy:.6e} 0.0\n")
        f.write(f"POINT_DATA {nx * ny}\n")
        f.write("SCALARS composition float 1\n")
        f.write("LOOKUP_TABLE default\n")
        for i in range(nx):
            for j in range(ny):
                f.write(f"{comp[i, j]:.6e}\n")

# --- Simulation Parameters ---

Nx, Ny = 64, 64          # Grid size
dx, dy = 1.0, 1.0        # Grid spacing
nstep = 2000             # Total number of time steps
nprint = 100             # Output frequency (used here for animation frame updates)
dtime = 1.0e-2           # Time step size
c0 = 0.40                # Initial composition
mobility = 1.0           # Mobility coefficient
grad_coef = 0.5          # Gradient energy coefficient

# --- Initialize the composition field (comp) ---
comp = initialize_microstructure(Nx, Ny, c0)

# --- Set up the plot for animation ---
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(comp, cmap='gray', origin='lower', vmin=0, vmax=1)
ax.set_title("Phase Field Evolution")
ax.axis("off")

# --- Define the update function for animation ---
def update(frame):
    global comp
    # Evolve the field for nprint steps per frame
    for _ in range(nprint):
        comp = evolve_phase_field(comp, mobility, grad_coef, dtime, dx, dy)
    im.set_data(comp)
    ax.set_title(f"Step: {frame * nprint}")
    return im,

# --- Create the animation ---
anim = FuncAnimation(fig, update, frames=int(nstep / nprint), blit=True, interval=100)

# Uncomment below if you wish to write VTK files instead of or in addition to animation:
# for step in range(1, nstep + 1):
#     comp = evolve_phase_field(comp, mobility, grad_coef, dtime, dx, dy)
#     if step % nprint == 0:
#         write_vtk(f"time_{step}.vtk", comp, dx, dy)

# Display the animation inline (in a Jupyter Notebook)
HTML(anim.to_html5_video())
